# connect to gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#zining
%cd /content/gdrive/MyDrive/SUTD/IRProject/

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/SUTD/IRProject/'
/content


In [ ]:
#luna
%cd /content/gdrive/MyDrive/IRProject/

/content/gdrive/.shortcut-targets-by-id/1ahfDdtcWq7L8op5-LpskyGKUqMpP6xZt/IRProject


# install anvil

In [ ]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


# Functions

In [ ]:
import pandas as pd
from ast import literal_eval
import time
from collections import OrderedDict, Counter
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from textblob import TextBlob, Word
import math
import sqlite3
import anvil.server

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import re
def no_punc(s):
    return re.sub("\W", " ", s).lower()

In [ ]:
stop_words = set(stopwords.words('english'))

def tokenize(ingredients_ls):
  recipe = []
  for ingre in ingredients_ls:
    ing = no_punc(ingre).split()
    recipe += ing
  ingre_lemma = []
  for word in recipe:
    w = Word(word)
    w_lemma = w.lemmatize()
    ingre_lemma.append(w_lemma)
  word_list = [word for word in ingre_lemma if word not in stop_words if not word.isdigit() if word != " "]
  return word_list

In [ ]:
print(tokenize(['apple cider', '2% milk', 'half-and-half', "acorns"]))

['apple', 'cider', 'milk', 'half', 'half', 'acorn']


In [ ]:
'''
A: Calculate TF
Given a tokenized document (a list of words), return a dictionary
with key as a word and value as the number of times it occurs in the doc. For
instance:
input: ['a', 'b', 'b', 'c', 'd']
output: {'a': 1, 'b': 2, 'c': 1, 'd': 1}
'''
def tf_(doc):
  frequencies = {}
  for term in doc:
    term_count = frequencies.get(term, 0) + 1
    frequencies[term] = term_count
  return frequencies

## Connection to database

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [ ]:
conn = create_connection("recipes.sqlite")

In [ ]:
conn.row_factory = sqlite3.Row  # creates dictionary cursor
cur = conn.cursor()
# cur.execute('SELECT * FROM RAW_recipes')
# r = cur.fetchall()
r=cur.execute("SELECT id,steps,ingredients FROM RAW_recipes where id=252171").fetchall()
r[0]['steps']

'[\'cook pasta according to package directions\', \'drain\', \'set aside\', \'in a large skillet cook the sausage , onion , and garlic until sausage is brown\', \'remove from skillet\', \'drain\', \'in a large bowl stir together pasta sauce , 3 / 4 cup of the mozzarella cheese , and the pepper\', \'add the cooked pasta and the drained sausage mixture\', \'stir gently to combine\', \'spoon the mixture into a 2-quart casserole\', \'bake , covered , in a 375f oven for 25 to 30 minutes or until nearly heated through\', \'uncover\', \'sprinkle with the remaining 1 / 4 cup mozzarella cheese\', \'bake about 5 minutes more or until cheese is melted\', "my card says it feeds 5 to 6 , and i say a family of four -- and a sad ol\'doggie -- for licking the casserole dish clean --"]'

# query retrieval



In [ ]:
def getQuery (userInput):
    userInput = userInput.lower().strip().split()
    cleanInput = []
    for x in userInput:
        x = x.strip(",.()&")
        cleanInput.append(x)
    cleanInput = tokenize(cleanInput)
    return cleanInput
    # for item in recipe_list:
    # id_no, doc = item.split('_')

def QueryRetrieval(cleanInput,inv_sorted):
    doc_list = []
    matching_recipe = []
    for i in range(len(cleanInput)):
        if cleanInput[i] in inv_sorted:
            matching_recipe = inv_sorted.get(cleanInput[i])
            doc_list.append(literal_eval(matching_recipe))

    total_doc_id_list = []
    for j in range(len(doc_list)):
        total_doc_id_list += doc_list[j]
    return (total_doc_id_list)
    print (total_doc_id_list)

# print(QueryRetrieval(getQuery(input("Enter all your ingredients: "))))
#create sample ingredient-to-id dataset to test
# (I suspect) might need to change something so that is looks at word and not letters

# check table/column names

In [ ]:
# check all tables
tabless = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tabless[0]

In [ ]:
# columnnames = cur.execute("Select * from inverted_index")
# columnnames = cur.execute("Select * from id_tf")
columnnames = cur.execute("Select * from doc_freq where ingredient='apple'")
columnnames.description
datarow = columnnames.fetchone()
for i in datarow:
  print(i)

apple
10516
3.09


# connect to anvil webapp

In [ ]:
# length of docs
conn = create_connection("recipes.sqlite")
cur = conn.cursor()
r=cur.execute("select count(*) from RAW_recipes").fetchone()
doc_len = r[0] # >>> 231636
doc_len 

231636

In [ ]:
sum_dflen = cur.execute("SELECT SUM(doc_freq) FROM doc_freq").fetchone()
sum_dflen # >>> 11792806
avg_doc_len = sum_dflen[0]/doc_len

In [ ]:
check_invidx = cur.execute('select * from inverted_index').fetchall()
invidx_dict = dict(check_invidx)

In [ ]:
# connect to webapp
anvil.server.connect('IDB6GE22DCR7MSYDBGU3FL76-3D63DXIVS3KTDKG3')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [ ]:
#this cell will keep running to be able to retrive info
@anvil.server.callable
def predict_recipe_final(ingred):
  k1=1.5
  b=0.75
  retrieved_recipes=[]
  startt=time.time()

  # database connection --> needs a new one inside this function if not anvil doesnt work
  conn3 = create_connection("recipes.sqlite")
  cur3 = conn3.cursor()
  
  # STEP 1: Get user input (MISSING: PREPROCESS QUERY to match corpus-->ingredients/steps)
  cleanInput = getQuery(ingred)
  print("Input:",cleanInput)

  # STEP 2: Retrieve Relevant Docs
  rel_docs = QueryRetrieval(cleanInput,invidx_dict)
  # print("relevant docs retrived:",rel_docs)
  rel_docs_cleaned = tf_(rel_docs) # combine the retrieved docs to remove duplicates. Eg. [doc1,doc4,doc4,doc5] --> {doc1:1,doc4:2,doc5:1}
  len_retreived = len(rel_docs_cleaned)
  print("Retrieved Docs: ",len_retreived)
  if len_retreived == 0:
    return 1

  # STEP 3: calculate bm25 score
  scores = []
  # retrieve the doc from corpus
  relevant_documents_id = str(list(rel_docs_cleaned.keys())).strip("[]")

  for query_term in cleanInput:
    if invidx_dict.get(query_term)==None:
      continue
    tb1=time.time()
    scoring_tfval={}
    doc = cur3.execute("select id,term_freq from id_tf where id in ({}) and term_freq like '%{}%' ".format(relevant_documents_id,query_term)).fetchall()
    term_idf = cur3.execute("select idf from doc_freq where ingredient like '%{}%'".format(query_term)).fetchone()

    tb3=time.time()
    for i in range(len(doc)):
      temp_tf = literal_eval(doc[i][1])
      try:
        scoring_tfval[doc[i][0]] = temp_tf[query_term]
      except KeyError:
        continue
    tb4=time.time()
    trythisdf = pd.DataFrame(data={'tf_value':scoring_tfval.values()},index=scoring_tfval.keys())
    trythisdf['numerator'] = trythisdf['tf_value'] + term_idf[0] * (k1 + 1)
    trythisdf['denominator'] = trythisdf['tf_value'] + k1 * (1 - b + b * doc_len / avg_doc_len)
    trythisdf['score'] = trythisdf['numerator']/trythisdf['denominator']
    tb2 = time.time()
    scores.append(dict(zip(trythisdf.index,trythisdf['score'])))
    print("pandas and sql took before scores zipping and append",tb2-tb1)
    print("pandas only took",tb2-tb4,'get tf part took',tb4-tb3,'query took',tb3-tb1)
  
  
  # save score_df
  scores_dict=Counter({})
  for s in scores:
    scores_dict+=Counter(s)
  scores_backtodict = dict(scores_dict)
  scores_tuple = list(scores_backtodict.items()) # (docID,score)
  
  sorted_scores = sorted(scores_tuple, key=lambda elem: elem[1] ,reverse=True)
  print(sorted_scores)
  endscoretime=time.time()
  
  # STEP 4: RANK the retrieved docs
  sorted_scores_id = str([i[0] for i in sorted_scores]).strip("[]")
  retrieved_recipes = cur3.execute("SELECT steps,name,ingredients FROM RAW_recipes where id in ({})" .format(sorted_scores_id)).fetchall()

  print("Top 10 Results: ",sorted_scores[:10])  #retrieved_recipes)
  endt = time.time()
  print("It took {:.3f} seconds to retreive and rank and {} seconds to sort.\n".format(endt-startt,endscoretime-tb2))
  return retrieved_recipes[:10]

anvil.server.wait_forever()

Input: ['apple', 'banana']
pandas and sql took before scores zipping and append 9.356173515319824
pandas only took 0.006869792938232422 get tf part took 2.057114362716675 query took 7.292189359664917
pandas and sql took before scores zipping and append 1.1359899044036865
pandas only took 0.005132436752319336 get tf part took 0.9113833904266357 query took 0.21947407722473145
[(140009, 0.005244865941915014), (415679, 0.0050497156505698365), (217960, 0.005049205367193405), (62564, 0.004855618532254683), (190858, 0.004855618532254683), (218907, 0.004855618532254683), (277746, 0.0048555968078538535), (65488, 0.004855205851112971), (105656, 0.004660880833096198), (183283, 0.004660880833096198), (404819, 0.004660880833096198), (324075, 0.004660826525248798), (213876, 0.00466078305311524), (9681, 0.004660620129560622), (114619, 0.004660620129560622), (32059, 0.004466088826090313), (45024, 0.004466088826090313), (58001, 0.004466088826090313), (71533, 0.004466088826090313), (103388, 0.0044660888

KeyboardInterrupt: ignored